Perform NicheNet analysis
================
Maria Solovey
2022-- DATE

In this notebook, we perform a basic NicheNet analysis on a Seurat object. We start by creating a Seurat object from the normalized count matrix and cell annotation file. You can find the notebooks to preprocessing, filtering and UMAP as well as the details of the input dataset in XXX. 


The used NicheNet custom networks, ligand-target matrix and expression
data of interacting cells can be downloaded from Zenodo. The NicheNet
networks and ligand-target matrix at [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3260758.svg)](https://doi.org/10.5281/zenodo.3260758)
and the raw single-cell data along with normalized and filtered at [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3531889.svg)](https://doi.org/10.5281/zenodo.3531889)


This pipeline is adapted from the offical tutorial of NicheNet's on their GitHub page. 

[Perform NicheNet analysis starting from a Seurat object: step-by-step analysis
](https://github.com/saeyslab/nichenetr/blob/master/vignettes/seurat_steps.md)


# Prepare NicheNet analysis

## Load required packages, read in the normalized count matrix of interacting cells and custom build of ligand-target prior model for NicheNet, ligand-receptor network and weighted integrated networks.

To identify potential ligand-receptor interactions between two cell populations, the NicheNet approach requires both the NicheNet ligand-receptor network and weighted networks. The ligand-target matrix provides information on the potential of certain ligands to regulate the expression of specific target genes, enabling prioritization of possible interactions based on observed gene expression effects through NicheNet's ligand activity analysis. This prioritization helps to infer the affected target genes of the selected ligands.

In [2]:
# libraries
library(nichenetr)
library(ggplot2)
library(Seurat)
library(tidyverse)
library(igraph)
require(circlize)
library(R.utils)
library(data.table) #to read gz file

The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)

Attaching SeuratObject

── Attaching core tidyverse packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ lubridate 1.9.2     ✔ tibble    3.2.1
✔ purrr     1.0.1     ✔ tidyr     1.3.0
── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

### Read in the expression data of interacting cells:
The dataset used here is publicly available single-cell data from XXX. The data was processed, and filtered by applying XXX. 

In [1]:
input_dir <- "../../../../../results/data_preprocessing/Similie/"
db_dir <- "../../../../../results/method_comparison/build_customDB/NicheNet/"
output_dir <- "../../../../../results/method_comparison/compare_algorithms/Similie/NicheNet/"

In [14]:
# # load counts
# print("load counts")
# counts <- read.table(gzfile(paste0(path_in,"/counts_corr.csv.gz")
#                             )
#                      ,sep = ","
#                      ,row.names = 1
#                      ,header = TRUE
#                      )
# # load counts

counts <- fread(paste0(input_dir,"counts_corr.csv.gz"), header = TRUE,check.names=FALSE)
counts <- as.data.frame(counts)
rownames(counts) <- counts$gene_symbol
counts <- counts[,-1]
# head(str(counts))
head(print(str(counts)))

'data.frame':	13044 obs. of  78518 variables:
 $ N10.EpiA.AACATTGAAAGTAG    : num  0 0 0 0 0 ...
 $ N10.EpiA.AACCCAGACGTAAC    : num  0 0 0 0 1.3 ...
 $ N10.EpiA.AATCTCACCGCATA    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ N10.EpiA.ACAATAACCGCTAA    : num  0 0 0 0 0.442 ...
 $ N10.EpiA.ACAATTGATGCAGT    : num  0 0 0 0 0.325 ...
 $ N10.EpiA.ACAGTGTGTTGGTG    : num  0 0 0 0 0 ...
 $ N10.EpiA.ACCGTGCTACTCAG    : num  0 1.57 0 0 0 ...
 $ N10.EpiA.ACGAGGGAACCTAG    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ N10.EpiA.ACGCAATGGTAGCT    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ N10.EpiA.ACTACGGAAAGAAC    : num  0 0 0 0 0 ...
 $ N10.EpiA.AGCGATACGGTAAA    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ N10.EpiA.AGTACGTGGAGGGT    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ N10.EpiA.ATAAGTACGGAGGT    : num  0 0 0 0 0.447 ...
 $ N10.EpiA.CAGCCTTGCGACTA    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ N10.EpiA.CCCACATGCCTTTA    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ N10.EpiA.CCCGATTGGCTGAT    : num  0 0 0 0 0.714 ...
 $ N10.EpiA.CGAACATGCCTCAC    : num  0 0 0 0 0 

NULL

In [15]:
# load cell annotation
print("load cell annotation")
anno_cells <- read.table(paste0(input_dir,"anno_cells_corr.txt")
                         ,sep = "\t"
                         ,row.names = 1
                         ,header = TRUE
                         )
head(print(str(anno_cells)))

[1] "load cell annotation"
'data.frame':	78518 obs. of  23 variables:
 $ cell_ID                      : chr  "N10.EpiA.AACATTGAAAGTAG" "N10.EpiA.AACCCAGACGTAAC" "N10.EpiA.AATCTCACCGCATA" "N10.EpiA.ACAATAACCGCTAA" ...
 $ cell_type_original           : chr  "Stem" "Stem" "Stem" "Stem" ...
 $ tissue_type                  : chr  "Epithelial" "Epithelial" "Epithelial" "Epithelial" ...
 $ cell_type                    : chr  "Epithelial" "Epithelial" "Epithelial" "Epithelial" ...
 $ region                       : chr  "Healthy" "Healthy" "Healthy" "Healthy" ...
 $ sample_ID                    : chr  "N10.A_Healthy" "N10.A_Healthy" "N10.A_Healthy" "N10.A_Healthy" ...
 $ health_status                : chr  "Healthy" "Healthy" "Healthy" "Healthy" ...
 $ total_counts_raw             : int  2995 6527 9262 13631 21207 6150 4101 11891 21496 22636 ...
 $ nr_genes                     : int  1119 1534 2042 2666 3290 1773 1203 2477 3121 3534 ...
 $ counts_upper_threshold       : int  30000 30000 30000 3

NULL

In [19]:
rownames(anno_cells) <- anno_cells$cell_ID

In [20]:
identical(rownames(anno_cells), colnames(counts))

[1] TRUE

In [21]:
srt=CreateSeuratObject(counts=counts, meta.data=anno_cells)

#### Read in ligand-target prior model, ligand-receptor network and weighted integrated networks:

In [22]:
ligand_target_matrix = readRDS(paste0(db_dir,"ligand_target_matrixWithweights.rds"))
lr_network = readRDS(paste0(db_dir,"lig_rec_sources.rds"))
weighted_networks = readRDS(paste0(db_dir,"weighted_networksWithSourceWeights.rds"))
weighted_networks_lr = weighted_networks$lr_sig %>% inner_join(lr_network %>% distinct(from,to), by = c("from","to"))
# weighted_networks_lr = weighted_networks_lr %>% filter(from %in% lr_network$from & to %in% lr_network$to)


In [23]:
srt@meta.data$health_status %>% table()

.
Colitis Healthy 
  39267   39251 

In [24]:
srt@meta.data$cell_type %>% table()

.
    B-cells Endothelial  Epithelial Fibroblasts Lymphocytes        Mast 
       6707        1778       23681        5978         447         834 
    Myeloid      Plasma     T-cells 
       3318       20563       15212 

In [25]:
Idents(srt) <- "cell_type"

In [26]:
cell_type_list <- unique(srt@meta.data$cell_type)
sample_list <- unique(srt@meta.data$sample_ID)

### 1. Define a “sender/niche” cell population and a “receiver/target”
In this case study, we consider each cell type as reciever as well as a sender in a for loop. We consider a gene to be expressed when it is expressed in at least 10% of cells in one cluster. We have 8 cell types, and each of them is tested as a sender as well as reciever. This allows us to check the cell types of sender and reciever later on during the interpretation. 

### 2. Define a gene set of interest
Here, the gene set of interest are the genes differentially expressed in each cell type after --AML diagnosis--. Thus, `AML` is the condition of interest, whereas the reference condition is annotated as `healthy`. The method to calculate the differential expression is here the standard Seurat Wilcoxon test. 

### 3. Define a set of potential ligands
These are ligands that are expressed by the sender cell population and bind a receptor expressed by the reciever population.

### 4. Perform NicheNet ligand activity analysis

Once NicheNet outputs which ligands from one or more cell population(s) (“sender/niche”) are most likely to affect target gene expression in an interacting cell population (“receiver/target”), we extract the pairs from the `weighted_networks_lr`

In [27]:
# Create an empty list to store the expressed genes for each sample and cell type
expressed_genes_list <- list()

# Iterate through each sample in the sample_list
for (sample in sample_list){
    
    # Subset the Seurat object to the current sample
    sample_subset <- subset(x = srt, subset = sample_ID == sample)
    
    # Get a list of unique cell types in the current sample
    cell_types <- unique(sample_subset@meta.data$cell_type)
    
    # Iterate through each cell type in the current sample
    for (cell_type in cell_types){
        
        # Call the function 'get_expressed_genes' to get the genes that are expressed in the current cell type
        expressed_genes <- get_expressed_genes(cell_type, sample_subset, 0.10)
        
        # Convert the expressed genes to a vector and remove duplicates
        expressed_genes <- expressed_genes %>% unlist() %>% unique()
        
        # Add the list of expressed genes for the current sample and cell type to the 'expressed_genes_list'
        expressed_genes_list[[sample]][[cell_type]] <- expressed_genes
        
    }
}


In [28]:
sample_subset

An object of class Seurat 
13044 features across 2693 samples within 1 assay 
Active assay: RNA (13044 features, 0 variable features)

In [29]:
length(expressed_genes_list[[sample]][[cell_type]])

[1] 3368

In [30]:
# Create an empty list to store the differential expression tables for each cell type
DE_tables <- list()

# Iterate through each cell type in the cell_type_list
for (cell_type in cell_type_list){
    
    # Subset the Seurat object to the current cell type
    seurat_subset <- subset(srt, idents = cell_type)
    
    # Set the identity of the cells in the subset to the 'health_status' column
    seurat_subset <- SetIdent(seurat_subset, value = seurat_subset[["health_status"]])
    
    # Set the conditions of interest and reference for the differential expression analysis
    condition_oi <- "Colitis"
    condition_reference <- "Healthy"
    
    # Call the function 'FindMarkers' to perform differential expression analysis and get the differential expression table
    DE_table <- FindMarkers(object = seurat_subset, ident.1 = condition_oi, 
                                    ident.2 = condition_reference, min.pct = 0.10) %>% rownames_to_column("gene")
    
    # Add the differential expression table for the current cell type to the 'DE_tables' list
    DE_tables[[cell_type]] <- DE_table
}


This function is designed to transform a data frame into a new data frame in wide format, with the same structure as the output from a community tool. The reason for this is to ensure consistency in the structure of the results table, which can facilitate further analysis or comparison with other data sets.

In particular, the input data frame is expected to have columns named "interaction", "sample", and "weight", representing interactions between entities in different samples. The function pivots the data frame from long format to wide format, with "interaction" as the ID column, "sample" as the column names, and "weight" as the values.

The resulting data frame has the same number of rows as the input data frame, but the number of columns is equal to the number of unique values in the "sample" column. This structure is consistent with the output of a community tool, which typically provides a table of interactions with each sample represented as a separate column.

In [31]:
# Define a function 'transform_interaction_df' that takes a data frame as input
transform_interaction_df <- function(df) {
  
  # Pivot the data frame to wide format, with 'interaction' as the ID column, 
  # 'sample' as the column names, and 'weight' as the values
  df_new <- df %>%
    pivot_wider(
      id_cols = interaction_ID,
      names_from = sample,
      values_from = c("weight")
    ) %>%
    
    # Reorder the columns to have 'interaction' as the first column
    select(interaction_ID, everything()) 
  
  # Return the pivoted data frame
  return(df_new)
}


In [17]:
# Get unique ligands and receptors from lr_network

ligands_in_db = lr_network %>% pull(from) %>% unique()
receptors_in_db = lr_network %>% pull(to) %>% unique()

# Initialize an empty list to store results
results=list()
ligand_activities_list <- list()
lr_network_df_large_full <- list()


for (receiver in cell_type_list){
    
    # Get DE genes for the receiver cell type
    geneset_oi <- DE_tables[[receiver]] %>% filter(p_val_adj <= 0.05 & abs(avg_log2FC) >= 0.25) %>% pull(gene)
    
#     n_DE_genes <- length(geneset_oi)
    
    # Iterate over each sample
    for (sample in names(expressed_genes_list)){
        
        # Get expressed genes for the receiver cell type in the current sample
        expressed_genes_receiver <- expressed_genes_list[[sample]][[receiver]]
        
        
        # Get the intersection of DE genes and expressed genes for the receiver cell type in the current sample
        geneset_oi_sample_expressed <- geneset_oi[geneset_oi %in% expressed_genes_receiver]
        
        # Get the intersection of DE genes and ligand-target gene matrix rows
        geneset_oi_sample_expressed <- geneset_oi_sample_expressed[geneset_oi_sample_expressed %in% rownames(ligand_target_matrix)]
        
        # Get the intersection of expressed genes for the receiver cell type in the current sample and ligand-target gene matrix rows
        background_expressed_genes  <- expressed_genes_receiver[expressed_genes_receiver %in% rownames(ligand_target_matrix)]
        
        # Iterate over each sender cell type
        for (sender in names(expressed_genes_list[[sample]])){
            
            
#             de_rec <- geneset_oi
#             de_lig <- DE_tables[[sender]] %>% filter(p_val_adj <= 0.05 & abs(avg_log2FC) >= 0.25) %>% pull(gene)
            
            
            # Get expressed genes for the sender cell type in the current sample
            expressed_genes_sender <- expressed_genes_list[[sample]][[sender]]
            
            # Check if the sender cell type is not null for the receiver cell type
            if (!is.null(expressed_genes_list[[sample]][[receiver]])){
                
                # Get the intersection of expressed ligands and ligands in the lr_network
                expressed_ligands <- expressed_genes_sender[expressed_genes_sender %in% ligands_in_db]
                expressed_receptors <- expressed_genes_receiver[expressed_genes_receiver %in% receptors_in_db]
                
                
                
                # Get the potential ligands based on the expressed ligands and receptors
                potential_ligands = lr_network %>% 
                    filter(from %in% expressed_ligands & to %in% expressed_receptors) %>% 
                        pull(from) %>% unique()
                
                
                
                potential_ligands = potential_ligands[potential_ligands %in% colnames(ligand_target_matrix)]
                
                
                # Predict ligand activities with NicheNet
                ligand_activities = predict_ligand_activities(geneset = geneset_oi_sample_expressed, 
                                                              background_expressed_genes = background_expressed_genes, 
                                                              ligand_target_matrix = ligand_target_matrix, 
                                                              potential_ligands = potential_ligands)
                
                # Arrange ligand activities by pearson correlation coefficient and add a rank column
                ligand_activities = ligand_activities %>% arrange(-pearson) %>% mutate(rank = rank(desc(pearson)))
                
                
                # Get the potential ligands based on the ligand activities
                potential_ligands = ligand_activities %>% pull(test_ligand) %>% unique()
                
#                 potential_ligands = intersect(potential_ligands, de_lig)
                
                
#                 best_upstream_ligands = ligand_activities %>% top_n(20, pearson) %>% arrange(-pearson) %>% pull(test_ligand) %>% unique()

                
                ligand_activities = ligand_activities %>% arrange(-aupr_corrected) %>% mutate(rank = rank(desc(aupr_corrected)))
                
                best_upstream_ligands = ligand_activities %>% top_n(30, aupr_corrected) %>% arrange(-aupr_corrected) %>% pull(test_ligand) %>% unique()
                
                # Filter the lr_network to only include interactions between potential ligands and expressed receptors
                lr_network_expressed = lr_network %>% 
                    filter(from %in% best_upstream_ligands & to %in% expressed_receptors) %>% distinct(from,to)
                
                
                # Get the corresponding interactions from the weighted network
                lr_network_df_large = weighted_networks_lr %>% filter(from %in% best_upstream_ligands & 
                                                                      to %in% expressed_receptors)
                
                # Add metadata to the network dataframe
                lr_network_df_large$sample <- sample
                lr_network_df_large$cell_type_l <- sender
                lr_network_df_large$cell_type_r <- receiver
                lr_network_df_large$pair <- paste(lr_network_df_large$from,lr_network_df_large$to, sep="_")
                lr_network_df_large$interaction_ID = paste0(lr_network_df_large$cell_type_l,":",
                                                         lr_network_df_large$from,"_",
                                                         lr_network_df_large$cell_type_r,":",
                                                         lr_network_df_large$to)
                
                
                df_new <- transform_interaction_df(lr_network_df_large)
                results[[sample]] <- rbind(results[[sample]],df_new)
                ligand_activities_list[[sample]][[sender]][[receiver]] <- ligand_activities
                
                lr_network_df_large_full[[sample]][[sender]][[receiver]] <- lr_network_df_large
#                 print(receiver)
#                 print(nrow(lr_network_df_large))
#                 write.csv(ligand_activities, paste("results6/",sample,"_",sender,"_",receiver,"_ligand_activities.csv", sep=""))
#                 write.csv(lr_network_df_large, paste("results6/",sample,"_",sender,"_",receiver,"_LR.csv", sep=""))
            }
        }
        
    }
    
}

Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warn

Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warn

Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warn

Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warn

Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warn

...

Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warn

Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warn

Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warn

Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”
Warning message in evaluate_target_prediction(setting, ligand_target_matrix, ligands_position):
“all target gene probability score predictions have same value”
Warn

Warning message in cor(prediction, response):
“the standard deviation is zero”
Warning message in cor(prediction, response, method = "s"):
“the standard deviation is zero”


In [18]:
matrix_result1 <- Reduce(
  
  # The `Reduce()` function takes two arguments: a function and a list.
  # In this case, the function is an anonymous function defined using the `function()` keyword.
  # This function takes two arguments `x` and `y` and performs a full join between them using the `full_join()` function from the `dplyr` package.
  # The `by = "interaction"` argument specifies that the join should be performed on the "interaction" column.
  function(x, y) full_join(x, y, by = "interaction_ID"), 
  
  # The second argument to the `Reduce()` function is a list called `results`.
  # This list contains data frames that need to be joined together.
  results
)

In [19]:
# Define a variable called `result` that will hold the output of the Reduce function
matrix_result <- Reduce(
  
  # The `Reduce()` function takes two arguments: a function and a list.
  # In this case, the function is an anonymous function defined using the `function()` keyword.
  # This function takes two arguments `x` and `y` and performs a full join between them using the `full_join()` function from the `dplyr` package.
  # The `by = "interaction"` argument specifies that the join should be performed on the "interaction" column.
  function(x, y) full_join(x, y, by = "interaction_ID"), 
  
  # The second argument to the `Reduce()` function is a list called `results`.
  # This list contains data frames that need to be joined together.
  results
)


In [20]:
# Example list of strings
strings <- matrix_result$interaction_ID

# Initialize empty vectors for each column
sender_celltype <- c()
sender_gene <- c()
receiver_celltype <- c()
receiver_gene <- c()

# Loop through each string and split it
for (string in strings) {
  parts <- strsplit(string, "_")
  
  # Split the sender part
  sender_parts <- strsplit(parts[[1]][1], ":")
  sender_celltype <- c(sender_celltype, sender_parts[[1]][1])
  sender_gene <- c(sender_gene, sender_parts[[1]][2])
  
  # Split the receiver part
  receiver_parts <- strsplit(parts[[1]][2], ":")
  receiver_celltype <- c(receiver_celltype, receiver_parts[[1]][1])
  receiver_gene <- c(receiver_gene, receiver_parts[[1]][2])
  interaction_ID <- c(string)
}

# Create a dataframe with the splitted values
df <- data.frame(
  sender_celltype = sender_celltype,
  sender_gene = sender_gene,
  receiver_celltype = receiver_celltype,
  receiver_gene = receiver_gene
)



In [21]:
# add interaction ID at the first column
df <- tibble::add_column(df, interaction_ID = matrix_result$interaction_ID, .before = "sender_celltype")

In [22]:
write.csv(matrix_result, paste0(output_dir,"NicheNet_scores.csv"))

In [23]:
write.csv(df, paste0(output_dir,"NicheNet_anno_interactions.csv"))

In [85]:
# int_ID <- matrix_result1$interaction_ID

# sender_celltype <- c()
# sender_gene <- c()
# receiver_celltype <- c()
# receiver_gene <- c()

# # Loop through each string and split it
# for (string in int_ID) {
#   parts <- strsplit(string, "_")
  
#   # Split the sender part
#   sender_parts <- strsplit(parts[[1]][1], ":")
#   sender_celltype <- c(sender_celltype, sender_parts[[1]][1])
#   sender_gene <- c(sender_gene, sender_parts[[1]][2])
  
#   # Split the receiver part
#   receiver_parts <- strsplit(parts[[1]][2], ":")
#   receiver_celltype <- c(receiver_celltype, receiver_parts[[1]][1])
#   receiver_gene <- c(receiver_gene, receiver_parts[[1]][2])
# }

# # Create a dataframe with the each compo
# df <- data.frame(
#   sender_celltype = sender_celltype,
#   sender_gene = sender_gene,
#   receiver_celltype = receiver_celltype,
#   receiver_gene = receiver_gene
# )



In [33]:
# lr_network$pair=paste0(lr_network$from, "_", lr_network$to)